In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')


c:\Users\jkoro\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

# Stałe
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
MAX_LEN = 128
TRAIN_SIZE = 0.8


In [4]:

# Wczytanie i przygotowanie danych
df = pd.read_csv('data/annotations_all_batches - WORD - SECOND BATCH.csv')
print(df.head())
df = df.fillna(method='ffill')

# Grupowanie po sentence_id
sentences = df.groupby('sentence_id').agg({
    'word': lambda x: list(x),
    'final-annotation': lambda x: list(x)
}).reset_index()


   sentence_id  word_id        word  Olek  Kuba Zgodne?  Stachu  \
0            1        1          Do     3     3       T     NaN   
1            1        2       Bosch     1     1       T     NaN   
2            1        3  SMV53L10EU     1     1       T     NaN   
3            1        4      pasuje     2     2       T     NaN   
4            1        5    IDEALNIE     2     2       T     NaN   

   final-annotation  Unnamed: 8  
0                 3         NaN  
1                 1         NaN  
2                 1         NaN  
3                 2         NaN  
4                 2         NaN  


In [7]:

# Klasa dataset
class TokenClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        words = self.texts[idx]
        labels = self.labels[idx]
        
        # Tokenizacja
        encoding = self.tokenizer(
            words,
            is_split_into_words=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Dostosowanie etykiet do tokenów
        word_ids = encoding.word_ids()
        label_ids = []
        
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)
            else:
                label_ids.append(labels[word_id])
                
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label_ids)
        }


In [8]:

# Przygotowanie danych
texts = sentences['word'].values
labels = sentences['final-annotation'].values

# Podział na zbiór treningowy i testowy
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, train_size=TRAIN_SIZE, random_state=42
)

# Inicjalizacja tokenizera i modelu
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained(
    'bert-base-multilingual-cased',
    num_labels=4
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:

# Przygotowanie datasetów
train_dataset = TokenClassificationDataset(train_texts, train_labels, tokenizer, MAX_LEN)
test_dataset = TokenClassificationDataset(test_texts, test_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [28]:

# Trening modelu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

for epoch in range(500):
    model.train()
    total_loss = 0
    
    for batch in train_loader:
        optimizer.zero_grad()
        
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch + 1}, Loss: {total_loss/len(train_loader)}')


Epoch 1, Loss: 0.017761006951332092
Epoch 2, Loss: 0.0023670087102800608
Epoch 3, Loss: 0.014826121740043163
Epoch 4, Loss: 0.014994713477790356
Epoch 5, Loss: 0.014142493717372417
Epoch 6, Loss: 0.008848750032484531
Epoch 7, Loss: 0.0019024789799004793
Epoch 8, Loss: 0.00822372641414404
Epoch 9, Loss: 0.0029362363275140524
Epoch 10, Loss: 0.004190526902675629
Epoch 11, Loss: 0.003527386812493205
Epoch 12, Loss: 0.007776548620313406
Epoch 13, Loss: 0.014850655570626259
Epoch 14, Loss: 0.004815701860934496
Epoch 15, Loss: 0.002422575606033206
Epoch 16, Loss: 0.0024573965929448605
Epoch 17, Loss: 0.0021067659836262465
Epoch 18, Loss: 0.002867379691451788
Epoch 19, Loss: 0.00442637549713254
Epoch 20, Loss: 0.003954277373850346
Epoch 21, Loss: 0.004102859180420637
Epoch 22, Loss: 0.004904411733150482
Epoch 23, Loss: 0.0022475607693195343
Epoch 24, Loss: 0.00214527384378016
Epoch 25, Loss: 0.0014689164236187935
Epoch 26, Loss: 0.0031625539995729923
Epoch 27, Loss: 0.0012902760645374656
Epoc

In [32]:

# Ewaluacja
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        preds = torch.argmax(outputs.logits, dim=2)
        
        for i in range(len(preds)):
            pred = preds[i][batch['attention_mask'][i] == 1]
            label = labels[i][batch['attention_mask'][i] == 1]
            
            pred = pred[label != -100]
            label = label[label != -100]
            
            predictions.extend(pred.cpu().numpy())
            true_labels.extend(label.cpu().numpy())

# Wyświetlenie wyników dla zbioru testowego
print("\nWyniki klasyfikacji:")
print(classification_report(true_labels, predictions))



Wyniki klasyfikacji:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        34
           1       0.58      0.68      0.63        56
           2       0.04      0.20      0.06         5
           3       0.69      0.82      0.75        11

    accuracy                           0.45       106
   macro avg       0.33      0.42      0.36       106
weighted avg       0.38      0.45      0.41       106



In [33]:

# Predykcje dla przykładowych zdań ze zbioru testowego
label_mapping = {0: 'negatywny', 1: 'neutralny', 2: 'pozytywny', 3: 'inne'}

def predict_sentence(sentence_words):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(
            sentence_words,
            is_split_into_words=True,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=MAX_LEN
        ).to(device)
        
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=2)
        
        word_predictions = []
        word_ids = inputs.word_ids()
        
        current_word = None
        current_predictions = []
        
        for token_idx, word_idx in enumerate(word_ids):
            if word_idx is None:
                continue
            if word_idx != current_word:
                if current_word is not None:
                    # Wybierz najczęstszą predykcję dla słowa
                    word_predictions.append(max(set(current_predictions), key=current_predictions.count))
                current_word = word_idx
                current_predictions = []
            current_predictions.append(predictions[0][token_idx].item())
        
        # Dodaj ostatnie słowo
        if current_predictions:
            word_predictions.append(max(set(current_predictions), key=current_predictions.count))
            
        return word_predictions


In [34]:

print("\nPrzykładowe predykcje dla zdań ze zbioru testowego:")
for i in range(min(3, len(test_texts))):  # Pokazujemy pierwsze 3 zdania
    sentence = test_texts[i]
    predictions = predict_sentence(sentence)
    
    print(f"\nZdanie {i+1}:")
    for word, pred in zip(sentence, predictions):
        pred_label = label_mapping[pred]
        print(f"Słowo: {word:15} Predykcja: {pred_label}")


Przykładowe predykcje dla zdań ze zbioru testowego:

Zdanie 1:
Słowo: Jakość          Predykcja: pozytywny
Słowo: i               Predykcja: inne
Słowo: praktyczność    Predykcja: pozytywny
Słowo: wykonania       Predykcja: pozytywny
Słowo: tego            Predykcja: inne
Słowo: trymera         Predykcja: neutralny
Słowo: pozostawia      Predykcja: pozytywny
Słowo: naprawdę        Predykcja: pozytywny
Słowo: wiele           Predykcja: pozytywny
Słowo: do              Predykcja: inne
Słowo: życzenia        Predykcja: neutralny
Słowo: O               Predykcja: inne
Słowo: golarce         Predykcja: neutralny
Słowo: w               Predykcja: inne
Słowo: tym             Predykcja: inne
Słowo: zestawie        Predykcja: neutralny
Słowo: nie             Predykcja: neutralny
Słowo: warto           Predykcja: neutralny
Słowo: nawet           Predykcja: neutralny
Słowo: wspominać       Predykcja: neutralny
Słowo: Lepiej          Predykcja: neutralny
Słowo: od              Predykcja: inne
Sło